In [ ]:
import pandas as pd
import tabula
import PyPDF2
import datetime  # Biblioteca para manejar fechas y horas
import xlsxwriter
import os
import re

In [ ]:
# Rutas de la carpeta con los PDFs y el template para extraer datos
ruta_carpeta = 'D:/DOCUMENTOS/Banco/Resumenes Bancos PDF/'
ruta_guardar = 'D:/DOCUMENTOS/Banco/Resumenes Bancos Excel/'
template_sin = 'Template_Bco_Patagonia.json'
template_cbu = 'Template_Bco_Patagonia_cbu.json'

In [ ]:
# Lista de títulos para identificar diferentes secciones en los extractos bancarios
Titulos = [
    'CUENTA CORRIENTE EN PESOS', 
    'CAJA DE AHORROS PESOS',
    'DEPOSITOS A PLAZO FIJO', 
    'PATAGONIA 24 - COMPRAS VISA ELECTRON', 
    'DEBITOS AUTOMATICOS REALIZADOS',
    'DEBITOS AUTOMATICOS  RECHAZADOS', 
    'TRANSFERENCIAS RECIBIDAS', 
    'TRANSFERENCIAS ENVIADAS Y ACEPTADAS', 
    'TARJETAS DE CREDITO',
    'SALDOS PENDIENTES DE COBRO', 
    'DETALLE - COMISION DE PAQUETES / MANTENIMIENTO',
    'SITUACION IMPOSITIVA'
]

# Lista de títulos sin espacios para facilitar el procesamiento
encontrar_list = [elem.replace(" ", "") for elem in Titulos]  

In [ ]:
# Definición de columnas para cada tabla en el PDF (usado por Tabula)
cols_cta_cte            = [12, 36, 44.5, 55, 70, 84.5]
cols_caja_ahorro        = [12, 36, 44.5, 55, 70, 84.5]
cols_plazo_fijo         = [12, 20, 24, 28, 36, 50, 56, 60, 68, 76, 80]
cols_compra_visa        = [11, 23, 27, 30, 34, 36, 55, 60, 70, 82, 90]
cols_debitos_realizados = [11, 28, 36, 56, 72, 88, 91, 94]
cols_debitos_rechazados = [11, 23, 31, 47, 58, 78, 88, 91, 94]
cols_transf_recibidas   = [11, 23, 30, 37, 49, 64, 76, 80, 90, 92, 95]
cols_transf_enviadas    = [11, 28, 36, 48, 70, 73, 86, 91, 94]
cols_saldos_pendiente   = [24, 28, 34, 42, 52, 60, 69, 74, 80, 83, 85]

In [ ]:
# Definición de los encabezados de cada tabla extraída de los PDFs
Encabezado_cta_cte            = ["Fecha", "Concepto", "Refer.", "Fecha Valor", "Debitos", "Creditos", "Saldo"]
Encabezado_caja_ahorro        = ["Fecha", "Concepto", "Refer.", "Fecha Valor", "Debitos", "Creditos", "Saldo"]
Encabezado_plazo_fijo         = ["Fecha", "Operacion", "Ren.", "Suc", "Tipo Cta.", "Capital", "Tasa", "Tipo", "Plazo", "Vcto.", "Mon.", "Monto al Vcto."]
Encabezado_compra_visa        = ["Fecha", "Tarjeta",  "Tipo",  "MDA" , "SBCTA", "Suc", "Nombre Comercio", "Refer.",  "Concepto", "Compra", "Extraccion", "Imp.T"]
Encabezado_debitos_realizados = ["Fecha", "Empresa", "Servicio", "Referencia", "Factura", "Importe", "Tipo" ,"Suc", "SCTA"]
Encabezado_debitos_rechazados = ["Fecha", "Empresa", "Servicio", "Referencia", "Factura", "Motivo", "Importe", "Tipo" ,"Suc", "SCTA"]
Encabezado_transf_recibidas   = ['Fecha', "Nombre Orig.", "Doc. Orig.", "Servicio", "Referencia", "CBU Origen", "Banco", "Suc Banc", "Imp. Original", "Tipo", "Mon", 'Suc']
Encabezado_transf_enviadas    = ['Fecha', "Nombre/Doc Destino", "Servicio", "Referencia", "CBU Destino", "Mon", "Importe Original", "Tipo", "Mon Cta.", 'Suc']
Encabezado_saldos_pendiente   = ["Concepto", "MDA", "Fecha", "Operacion", "Importe", "Bonif.", "Saldo Pendiente", "Tipo", "Cuenta", "Suc", "MDA Cta.", "SCTA"]

In [ ]:
# Función para buscar palabras clave en el texto de un archivo PDF
def buscar_en_pdf(pdf_path, palabras_clave):
    paginas_con_coincidencias = {}
    # Abrimos el archivo PDF
    with open(pdf_path, 'rb') as archivo_pdf:
        lector_pdf = PyPDF2.PdfReader(archivo_pdf)
        numero_paginas = len(lector_pdf.pages)
        # Iteramos sobre cada página del PDF
        for num_pagina in range(numero_paginas):
            pagina = lector_pdf.pages[num_pagina]
            texto = pagina.extract_text()
            # Buscamos cada palabra clave en el texto de la página
            for palabra in palabras_clave:
                if palabra.lower() in texto.lower():  # Búsqueda insensible a mayúsculas/minúsculas
                    if palabra not in paginas_con_coincidencias:
                        paginas_con_coincidencias[palabra] = []
                    paginas_con_coincidencias[palabra].append(num_pagina + 1)  # +1 para ser 1-indexado
    # Convertimos el resultado en un DataFrame para manejarlo mejor
    df_paginas = pd.DataFrame(list(paginas_con_coincidencias.items()), columns=['Titulo', 'Paginas'])
    df_paginas = df_paginas.explode('Paginas').reset_index(drop=True)
    return df_paginas

In [ ]:
# Función para identificar el grupo al que pertenece una fila en función de las palabras clave
def encontrar_grupo(row):
    for elem in encontrar_list:
        if elem in row['concatenated']:
            return elem
    return None 

In [ ]:
# Función para procesar el contenido de las páginas del PDF según el título
def procesar_pdf (archivo_pdfs, paginas, delimitadores, Encabezado, titulo, planilla):
    # Usamos Tabula para leer las tablas en base a un template predefinido
    origen_pdf = tabula.read_pdf_with_template(
        archivo_pdfs, 
        pages = paginas,
        template_path = planilla,
        relative_columns=True,
        columns = delimitadores,
        stream=True,
        pandas_options={'header': None},
        silent=True
    )
    # Renombrar las columnas con los encabezados correctos
    df_renombrados = [df.rename(columns=dict(zip(df.columns, Encabezado))) for df in origen_pdf]
    df = pd.concat(df_renombrados, ignore_index=True)

    # Concatenar todas las columnas de cada fila para facilitar la búsqueda de palabras clave
    df['concatenated'] = df.apply(lambda row: ''.join(str(val).replace(' ',"") for val in row), axis=1)
    df['concatenated'] = df['concatenated'].fillna('').astype(str)

    # Aplicar condición para agregar la columna CBU solo si el título es "AAAA" o "BBBB" 
    if titulo in ["CUENTACORRIENTEENPESOS", "CAJADEAHORROSPESOS"]: 
        df['CBU'] = df['concatenated'].str.extract(r'CBU:([0-9]{22})') # Usar str.extract con una expresión regular para extraer el valor de CBU 
        df['CBU'] = df['CBU'].ffill() # Forward fill en caso de valores faltantes
    
    # Asignar el grupo correcto a cada fila
    df['Grupo'] = df.apply(encontrar_grupo, axis=1)
    df['Grupo'] = df['Grupo'].ffill()  # Forward fill en caso de valores faltantes

    # Filtrar las filas que coinciden con el título actual
    df_filter = df[df['Grupo'] == titulo]

    # Filtrar filas que contengan fechas válidas
    patron = r'\d{1,2}/\d{1,2}/\d{2,4}'
    df_filter = df_filter[df_filter['Fecha'].str.contains(patron, na=False, regex=True)]

    # Eliminar columnas innecesarias
    df_tbl = df_filter.drop(columns=['concatenated', 'Grupo'])

    return df_tbl

In [ ]:
# Función para identificar las páginas de un PDF que contienen un determinado título
def paginas_pdf(df_paginas, titulo):
    fila_cta_cte = df_paginas[df_paginas['Titulo'].str.contains(titulo, case=False)].index.to_list()[0]
    page_inicio = df_paginas.iloc[fila_cta_cte, 1]
    page_fin = df_paginas.iloc[fila_cta_cte+1, 1]
    select_pages = list(range(page_inicio, page_fin + 1))
    return select_pages

In [ ]:
# Funcion: Obtener cuit del resumen pdf
def obtener_cuit (ubicacion_pdf):
    pdf_cuit = tabula.read_pdf(ubicacion_pdf, pages=1,
                                relative_area=True, 
                                relative_columns=True, 
                                area=[23, 0, 30, 100],
                                columns=[50],
                                pandas_options={'header': None}, 
                                silent=True)
    fila = pdf_cuit[0].values[0]
    cuit = fila[0].split(' ')[1]
    return cuit

In [ ]:
# Listar todos los archivos PDF en la carpeta especificada
Lista_archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.pdf')]

In [ ]:
# Procesar cada archivo PDF
for archivo in Lista_archivos:
    pdf_archivo = os.path.join(ruta_carpeta, archivo)

    # Buscar las páginas que contienen los títulos
    df_paginas = buscar_en_pdf(pdf_archivo, Titulos)

    # Procesar cada sección de las transacciones y extraer los datos
    df_cta_cte            = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[0]), cols_cta_cte,            Encabezado_cta_cte, encontrar_list[0], template_cbu)            if any(Titulos[0] in elem for elem in df_paginas['Titulo']) else print('Cta. Cte: Siguiente no aplica al caso')
    df_cta_ahorro         = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[1]), cols_caja_ahorro,        Encabezado_caja_ahorro, encontrar_list[1], template_cbu)        if any(Titulos[1] in elem for elem in df_paginas['Titulo']) else print('Cta. Ahorro: Siguiente no aplica al caso')
    df_plazo_fijo         = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[2]), cols_plazo_fijo,         Encabezado_plazo_fijo, encontrar_list[2], template_sin)         if any(Titulos[2] in elem for elem in df_paginas['Titulo']) else print('Plazo Fijo: Siguiente no aplica al caso')    
    df_compra_visa        = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[3]), cols_compra_visa,        Encabezado_compra_visa, encontrar_list[3], template_sin)        if any(Titulos[3] in elem for elem in df_paginas['Titulo']) else print('Compras Visa: Siguiente no aplica al caso')
    df_dbo_realizados     = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[4]), cols_debitos_realizados, Encabezado_debitos_realizados, encontrar_list[4], template_sin) if any(Titulos[4] in elem for elem in df_paginas['Titulo']) else print('DEB realizados: Siguiente no aplica al caso')
    df_dbo_rechazados     = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[5]), cols_debitos_rechazados, Encabezado_debitos_rechazados, encontrar_list[5], template_sin) if any(Titulos[5] in elem for elem in df_paginas['Titulo']) else print('DEB rechazados: Siguiente no aplica al caso')
    df_transf_recicibidas = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[6]), cols_transf_recibidas,   Encabezado_transf_recibidas, encontrar_list[6], template_sin)   if any(Titulos[6] in elem for elem in df_paginas['Titulo']) else print('Transf recibidas: Siguiente no aplica al caso')
    df_transf_enviadas    = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[7]), cols_transf_enviadas,    Encabezado_transf_enviadas, encontrar_list[7], template_sin)    if any(Titulos[7] in elem for elem in df_paginas['Titulo']) else print('Transf enviadas: Siguiente no aplica al caso')
    df_saldo_pendiente    = procesar_pdf(pdf_archivo, paginas_pdf(df_paginas, Titulos[9]), cols_saldos_pendiente,   Encabezado_saldos_pendiente, encontrar_list[9], template_sin)   if any(Titulos[9] in elem for elem in df_paginas['Titulo']) else print('Saldos Pend: Siguiente no aplica al caso')

    # Lista de dataframes y nombres de las hojas en Excel
    dataframes_a_guardar = [
        (df_cta_cte, 'Cta_cte'),
        (df_cta_ahorro, 'Cta_ahorro'),
        (df_plazo_fijo, 'Plazo_fijo'),
        (df_compra_visa, 'Compra Visa'),
        (df_dbo_realizados, 'Debitos Realizados'),
        (df_dbo_rechazados, 'Debitos Rechazados'),
        (df_transf_recicibidas, 'Transf Recibidas'),
        (df_transf_enviadas, 'Transf Enviadas'),
        (df_saldo_pendiente, 'Saldos Pendientes')
    ]

    # Seleccionar Fecha desde y hasta
    desde = df_cta_cte['Fecha'].iloc[0]
    hasta = df_cta_cte['Fecha'].iloc[-1]
    # Convertimos las fechas a objetos datetime
    desde_fecha = datetime.datetime.strptime(desde, "%d/%m/%y")
    hasta_fecha = datetime.datetime.strptime(hasta, "%d/%m/%y")
    # Formateamos las fechas en el formato deseado
    date_name = f"{desde_fecha.strftime('%Y-%m-%d')} hasta {hasta_fecha.strftime('%Y-%m-%d')}"
    
    # Obtener el primer cuit del pdf
    cuit = obtener_cuit(pdf_archivo)
    
    # Crea un archivo de Excel para almacenar los datos procesados y ubicacion donde se guardara el archivo
    nombre_archivo = ruta_guardar + cuit +"_"+ date_name + '.xlsx'
    
    # Crear un archivo Excel con el nombre del archivo pdf        
    writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')

    # Guardar cada dataframe en una hoja separada
    for elemento, nombre_hoja in dataframes_a_guardar:
        if isinstance(elemento, pd.DataFrame):
            elemento.to_excel(writer, sheet_name=nombre_hoja, index=False)
        else:
            # Si no es un dataframe, crea una hoja especial para textos
            df_texto = pd.DataFrame({'Texto': [elemento]})
            df_texto.to_excel(writer, sheet_name=nombre_hoja, index=False)
    
    # Cerrar el archivo Excel y guardar los datos
    writer.close()
    print(f"Se han guardado los dataframes y textos en el archivo '{nombre_archivo}' en hojas separadas.")